In [23]:
import json
import pandas as pd


"""
TODO:
1. Data sudah bener2 post_rendered, tinggal processing untuk di training BERT
"""

def remove_backslash(text: str) -> str:
    result = text.replace("\\", " ")
    return result

def remove_redundant_white_spaces(text: str) -> str:
    from re import sub
    return sub(r'\s+', ' ', text).strip()


file_path = 'C:/wamp64/www/artofproblemsolving/data/data_post_rendered.csv'
df = pd.read_csv(file_path)

# display_dictionary(data)
